In [2]:
# get the data from the internet
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-06-16 18:00:42--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.005s  

2024-06-16 18:00:42 (236 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
# read the data
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [4]:
print ("length of the dataset in characters: ", len(text))

length of the dataset in characters:  1115394


In [5]:
print ("First 100 characters of the dataset: ", text[:100])

First 100 characters of the dataset:  First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print ("Number of unique characters in the dataset: ", vocab_size)
print (''.join(chars))

Number of unique characters in the dataset:  65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [9]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda x: [stoi[ch] for ch in x] # encoder: take a string, output a list of integers
decode = lambda x: ''.join([itos[i] for i in x]) # decoder: take a list of integers, output a string

print (encode('hello'))
print (decode([0, 1, 2, 3, 5]))


[46, 43, 50, 50, 53]

 !$'


In [10]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)

print (data.shape, data.dtype)
print (data[:100]) # first 100 characters

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [11]:
# training and validation split
n = int(0.9*len(data))
train_data, val_data = data[:n], data[n:]

In [12]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print (f"when the context is {context} the next character is {target}")

when the context is tensor([18]) the next character is 47
when the context is tensor([18, 47]) the next character is 56
when the context is tensor([18, 47, 56]) the next character is 57
when the context is tensor([18, 47, 56, 57]) the next character is 58
when the context is tensor([18, 47, 56, 57, 58]) the next character is 1
when the context is tensor([18, 47, 56, 57, 58,  1]) the next character is 15
when the context is tensor([18, 47, 56, 57, 58,  1, 15]) the next character is 47
when the context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the next character is 58


In [18]:
torch.manual_seed(1337)

batch_size = 4 # number of sequences in a mini-batch
block_size = 8 # number of characters in a sequence

def get_batch(split):
    # generate a small batch of data of input x and target y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print (xb.shape, yb.shape)
print(xb)
print (yb)

torch.Size([4, 8]) torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets = None):
        
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            # idx and targets are both (B,T) tensor of integers
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # B, C
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # B, C
            # sample from the distribution
            next_token = torch.multinomial(probs, num_samples=1) # B, 1
            # append sampled index to the running sequence
            idx = torch.cat([idx, next_token], dim=1)
        return idx
        
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print (logits.shape)
print (loss)


print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [31]:
# create a pytorch optimizer
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [35]:
batch_size = 32

for steps in range(10000):
    # get a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if steps % 100 == 0:
        print (f"step: {steps}, loss: {loss.item()}")

step: 0, loss: 3.0029618740081787
step: 100, loss: 3.0597994327545166
step: 200, loss: 2.9590420722961426
step: 300, loss: 2.982276201248169
step: 400, loss: 2.9200470447540283
step: 500, loss: 2.84088134765625
step: 600, loss: 2.8899765014648438
step: 700, loss: 2.9750688076019287
step: 800, loss: 2.808044672012329
step: 900, loss: 2.7770206928253174
step: 1000, loss: 2.747230291366577
step: 1100, loss: 2.6850526332855225
step: 1200, loss: 2.679885149002075
step: 1300, loss: 2.68688702583313
step: 1400, loss: 2.810159683227539
step: 1500, loss: 2.691971778869629
step: 1600, loss: 2.66461181640625
step: 1700, loss: 2.6310133934020996
step: 1800, loss: 2.7520360946655273
step: 1900, loss: 2.5809037685394287
step: 2000, loss: 2.629011869430542
step: 2100, loss: 2.624750852584839
step: 2200, loss: 2.547957181930542
step: 2300, loss: 2.58158540725708
step: 2400, loss: 2.6034939289093018
step: 2500, loss: 2.617574453353882
step: 2600, loss: 2.5722484588623047
step: 2700, loss: 2.51136636734

In [36]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))



Thilapo belll w anive? nanowalarpe JOFiase ig bo, t ced bere
LLe neset aiemy
I wie yat y mexirourou


In [2]:
# the mathematical trick in self attention

import torch
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [4]:
# We want x[t,t]=mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [5]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T,T) @ (B,T,C) = (B,T,C)
torch.allclose(xbow, xbow2)

False

In [19]:
# version 3: use softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = torch.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow2, xbow3)

True

In [26]:
# import nn
import torch
import torch.nn as nn
# version 4: self attention
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# single head
head_size=16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # B,T,H
q = query(x) # B,T,H
wei = q @ k.transpose(-1,-2) # (B, T, 16) @ (B, 16, T) = (B, T, T)


tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = torch.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])